In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Deep Learning Challenge

In [ ]:
from google.colab import files
files.upload()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from data_preparation import load_data, delete_outliers
from models import *
import numpy as np
from visualization import plot_history


X, y = load_data()

X, y, num_outliers = delete_outliers(X, y)

# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, stratify=np.argmax(y_train_val,axis=1))

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(np.unique(y_train, return_counts=True))
print(np.unique(y_val, return_counts=True))


models = []

models.append(build_custom_model(X_train.shape[1:], 0.001, tf.keras.Sequential([]), "Vanilla"))
models.append(build_custom_model(X_train.shape[1:], 0.001, tf.keras.Sequential([
            tfkl.RandomRotation(0.4),
        ]), "Rotation Augmentation"))
models.append(build_custom_model(X_train.shape[1:], 0.001, tf.keras.Sequential([
            tfkl.RandomZoom(0.2),
        ]), "Zoom Augmentation"))
models.append(build_transfer_model(X_train.shape[1:], 0.001))

for model in models:

    history = model.fit(
        x = X_train,
        y = y_train,
        batch_size = 32,
        epochs = 200,
        validation_data = (X_val, y_val),
        callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
    ).history


    plot_history(history, model.name)